# ENGSCI 700 Geothermal Reservoir Optimisation

This workbook is for extracting Contact well data and recreating the plots.

(Unix) launch with `cd src` >`jupyter notebook`

File structure: 
```
(root)
├── src
│    └── Python Test.ipynb
└── wairakei_data
     └── Liquid wells (version 1).xlsx
```

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Check if Excel file is already in memory (loading is slow)
try:
    xl
except:
    xl = pd.ExcelFile('../wairakei_data/Liquid wells (version 1).xlsx')
print(xl.sheet_names[:5], '...')

## Prepare data

In [ ]:
wells = ['wk255', 'wk256', 'w258', 'w259']  # wells to load data from

dfs = []
for well in wells:
    df = xl.parse(well)                                       # select well data
    df['well'] = well                                            # label data with well name
    dfs.append(df)
df = pd.concat(dfs)

# df = df.loc[:, ~df.columns.str.contains('^Unnamed|SUMMARY|slope|intercept')]     # remove extra columns
df = df[['date', 'whp', 'mf', 'h', 'well']]                      # instead, only keep certain columns
df['mf'] = pd.to_numeric(df['mf'], errors='coerce')              # remove 'dummy' entries
df = df.dropna(subset=['date', 'whp', 'mf'])                     # remove NA


df['date_numeric'] = (df['date'] - df['date'].min())  / np.timedelta64(1,'D') #  date_numeric = days since first
print(df.head())

## Create exploratory plots

In [ ]:
import itertools

cmap = plt.get_cmap('viridis')
fig, (ax1, ax2) = plt.subplots(1,2)

# left plot (not that useful tbh)
df.plot('date', 'whp', style='x', ax=ax1)
ax1.set_xlabel('date')
ax1.set_ylabel('whp')

# right plot
marker = itertools.cycle(['o', ',', '+', 'x', '*', '.'])
for well in wells:
    plt.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=next(marker), label=well)
ax2.set_xlabel('whp')
ax2.set_ylabel('mf')
plt.legend()

plt.show()

## Set up regression data and create prediction frame for plotting

In [ ]:
from itertools import product

# generate prediction dataframe for plotting
date_pred = np.linspace(np.min(df['date_numeric']), np.max(df['date_numeric']), 6)
whp_pred = np.linspace(0, 16, 1000)
well_pred = wells
pred = pd.DataFrame(list(product(date_pred, whp_pred, well_pred)), columns=['date_numeric', 'whp', 'well'])
print(pred.head())

## Perform regression and prediction

In [ ]:
from statsmodels.formula.api import ols

# Not conditioned on date
model1 = ols("mf ~ well * whp", data=df)
results1 = model1.fit()
pred['mf1'] = results1.predict(pred)

# Linear fit dependent on date
model2 = ols("mf ~ well * (whp + date_numeric)", data=df)
results2 = model2.fit()
pred['mf2'] = results2.predict(pred)

# Elliptic fit dependent on date
model3 = ols("np.power(mf,2) ~ well * (np.power(whp,2) + date_numeric)", data=df)
results3 = model3.fit()
pred['mf^2'] = results3.predict(pred)
pred.loc[pred['mf^2'] < 0, 'mf^2'] = np.nan    # remove invalid results
pred['mf3'] = np.sqrt(pred['mf^2'])

## Create plots

In [ ]:
# ===============================================================
# Set up axes
# ===============================================================

from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

# colors
indices = np.linspace(0, cmap.N, len(df))
my_colors = [cmap(int(i)) for i in indices]

# subplots
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=[9,4], gridspec_kw={'width_ratios': [9,9,9,1]})
ax1.get_shared_y_axes().join(ax1, ax2, ax3)
ax1.set_ylim([0, 1000])
ax1.set_title('$mf \sim whp$')
ax1.set_ylabel('Mass flow')
ax1.set_xlabel("Well head pressure")
ax1.set_xlim(5, 16)
ax2.set_title('$mf \sim whp + date$')
ax3.set_title('$mf^2 \sim whp^2 + date$')
for ax in [ax2, ax3]:
    ax.set_yticklabels([])
    ax.set_xlim(5, 16)

# create date colorbar
indices = np.linspace(0, cmap.N, len(date_pred))
my_colors = [cmap(int(i)) for i in indices]
norm = Normalize(np.min(df['date']).year, np.max(df['date']).year)
cb = ColorbarBase(ax4, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label('Year')

linestyles = itertools.cycle(('-', '--', '-.', ':'))

# ===============================================================
# Plot data
# ===============================================================

# plot data points
marker.__init__()
for well in wells:
    mkr = next(marker)
    for ax in [ax1, ax2, ax3]:
        ax.scatter('whp', 'mf', c='date_numeric', data=df.loc[df['well']==well], marker=mkr, label=well)
    
for well in wells:
    
    lty = next(linestyles)
    # model 1
    # 'data' argument filters the data to just the data from one well, using a single date
    ax1.plot('whp', 'mf1', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==np.min(pred['date_numeric']))])

    # model 2 & 3
    for i, date in enumerate(date_pred):
        # 'data' argument similar, for a specific prediction date in the loop
        ax2.plot('whp', 'mf2', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)], c=my_colors[i])
        ax3.plot('whp', 'mf3', lty, data=pred[(pred['well']==well) & (pred['date_numeric']==date)], c=my_colors[i])

# show model selection criteria
for ax, result in zip([ax1, ax2, ax3], [results1, results2, results3]):
    ax.legend(['Adj $R^2$: %.2f' % result.rsquared_adj,
               'AIC: %.2f' % result.aic], 
              handlelength=0, handletextpad=0, loc=1).legendHandles[0].set_visible(False)

In [ ]:
df.plot('date', 'h', style='o', alpha=0.5)
plt.show()